### This script will run over all files in `/data/game_info/` and write to csv files in `/data/csv`

## Configure input and output directories

In [ ]:
import os
import json
import pandas as pd
from pathlib import Path
import re
import concurrent.futures
import time
from threading import Lock
from tqdm import tqdm
import csv

# Input and output directories
input_dir = Path('../data/game_info')
output_dir = Path('../data/csv')

# Create output directory if it doesn't exist
output_dir.mkdir(parents=True, exist_ok=True)

## Create some utility functions and initialize lists

In [23]:
# Extract text plain from HTML
def strip_html(text):
    if not isinstance(text, str):
        return ""
    return re.sub(r'<.*?>', '', text)

# Clean text from problematic chars
def clean_text_for_csv(text):
    if not isinstance(text, str):
        return ""
    # Replace end of lines to whitespaces
    text = re.sub(r'[\r\n]+', ' ', text)
    # Remove control chars
    text = re.sub(r'[\x00-\x1F\x7F]', '', text)
    return text

# Shorten desctriptions
def short_description(text, max_length=200):
    if not isinstance(text, str):
        return ""
    text = strip_html(text)
    # Clean some chars
    text = clean_text_for_csv(text)
    if len(text) <= max_length:
        return text
    return text[:max_length-3] + "..."

# Initialize lists
games_data = []
platforms_data = []
genres_data = []
stores_data = []
developers_data = []
publishers_data = []
tags_data = []
ratings_data = []
status_data = []
metacritic_platforms_data = []
derived_metrics_data = [] 

# Lock for synchronization
data_lock = Lock()

## Create function to caclulate new derived metrics

In [24]:
# Funtion to get derived metrics
def calculate_derived_metrics(game, status):
    metrics = {
        'game_id': game.get('id'),
        'completability_index': 0.0,
        'consensus_score': 0.0,
        'time_rating_ratio': 0.0,
        'platform_diversity_index': 0.0,
        'acquisition_play_ratio': 0.0
    }
    
    # 1. Completability index: beaten / (beaten + dropped)
    beaten = status.get('beaten', 0)
    dropped = status.get('dropped', 0)
    if (beaten + dropped) > 0:
        metrics['completability_index'] = beaten / (beaten + dropped)
    
    # 2. Consensus score: merge metacritic and user reviews
    metacritic = game.get('metacritic', 0)
    user_rating = game.get('rating', 0) * 20  # Convert to same scale
    if metacritic and user_rating:
        # 50/50 between metacritic and users
        metrics['consensus_score'] = (metacritic + user_rating) / 2
    elif metacritic:
        metrics['consensus_score'] = metacritic
    elif user_rating:
        metrics['consensus_score'] = user_rating
    
    # 3. Time-Rating ratio: playtime / rating (if rating > 0)
    playtime = game.get('playtime', 0)
    rating = game.get('rating', 0)
    if rating > 0 and playtime > 0:
        metrics['time_rating_ratio'] = playtime / rating
    
    # 4. Platform diversity index: number of platforms
    platforms_count = len(game.get('platforms', []))
    if platforms_count > 0:
        metrics['platform_diversity_index'] = platforms_count
    
    # 5. Acquisition-Play ratio: (played + playing + beaten) / owned
    played = status.get('yet', 0) + status.get('playing', 0) + status.get('beaten', 0)
    owned = status.get('owned', 0)
    if owned > 0:
        metrics['acquisition_play_ratio'] = played / owned
    
    return metrics

## Function to process a JSON

In [25]:
# Function to process a single JSON
def process_json_file(json_file):
    try:
        # Load JSON
        with open(json_file, 'r', encoding='utf-8') as f:
            try:
                game = json.load(f)
            except json.JSONDecodeError:
                return f"Error: Error parsing JSON {json_file}"
                
        # Verify is a valid json
        if not isinstance(game, dict):
            return f"Error: File {json_file} doesn't have a valid JSON"
        
        # Get basic data
        game_id = game.get('id')
        
        if not game_id:
            return f"Warning: Not id found for file {json_file}, skipping file"
        
        # Local data for this file
        local_games_data = []
        local_platforms_data = []
        local_genres_data = []
        local_stores_data = []
        local_developers_data = []
        local_publishers_data = []
        local_tags_data = []
        local_ratings_data = []
        local_status_data = []
        local_metacritic_platforms_data = []
        local_derived_metrics_data = []
        
        # 1. Basic data
        local_games_data.append({
            'id': game_id,
            'slug': game.get('slug', ''),
            'name': clean_text_for_csv(game.get('name', '')),
            'released': game.get('released', ''),
            'metacritic': game.get('metacritic', None),
            'rating': game.get('rating', None),
            'rating_top': game.get('rating_top', None),
            'playtime': game.get('playtime', 0),
            'esrb_rating': clean_text_for_csv(game.get('esrb_rating', {}).get('name', '') if game.get('esrb_rating') else ''),
            'description_short': short_description(game.get('description', '')),
            'website': game.get('website', ''),
            'added': game.get('added', 0),
            'screenshots_count': game.get('screenshots_count', 0),
            'background_image': game.get('background_image', ''),
            'metacritic_url': game.get('metacritic_url', ''),
            'tba': game.get('tba', False),
            'updated': game.get('updated', '')
        })
        
        # 2. Platforms
        for platform_data in game.get('platforms', []) or []:
            if not isinstance(platform_data, dict):
                continue
                
            platform = platform_data.get('platform', {}) or {}
            if not isinstance(platform, dict):
                platform = {}
                
            requirements = platform_data.get('requirements', {}) or {}
            if not isinstance(requirements, dict):
                requirements = {}
                
            local_platforms_data.append({
                'game_id': game_id,
                'platform_id': platform.get('id', None),
                'platform_name': platform.get('name', ''),
                'platform_slug': platform.get('slug', ''),
                'released_at': platform_data.get('released_at', ''),
                'minimum_requirements': requirements.get('minimum', ''),
                'recommended_requirements': requirements.get('recommended', '')
            })
        
        # 3. Genres
        for genre in game.get('genres', []) or []:
            if not isinstance(genre, dict):
                continue
            local_genres_data.append({
                'game_id': game_id,
                'genre_id': genre.get('id', None),
                'genre_name': genre.get('name', ''),
                'genre_slug': genre.get('slug', '')
            })
        
        # 4. Stores
        for store_data in game.get('stores', []) or []:
            if not isinstance(store_data, dict):
                continue
            store = store_data.get('store', {}) or {}
            if not isinstance(store, dict):
                store = {}
            local_stores_data.append({
                'game_id': game_id,
                'store_id': store.get('id', None),
                'store_name': store.get('name', ''),
                'store_slug': store.get('slug', ''),
                'store_url': store_data.get('url', '')
            })
        
        # 5. Developers
        for developer in game.get('developers', []) or []:
            if not isinstance(developer, dict):
                continue
            local_developers_data.append({
                'game_id': game_id,
                'developer_id': developer.get('id', None),
                'developer_name': developer.get('name', ''),
                'developer_slug': developer.get('slug', '')
            })
        
        # 6. Publishers
        for publisher in game.get('publishers', []) or []:
            if not isinstance(publisher, dict):
                continue
            local_publishers_data.append({
                'game_id': game_id,
                'publisher_id': publisher.get('id', None),
                'publisher_name': publisher.get('name', ''),
                'publisher_slug': publisher.get('slug', '')
            })
        
        # 7. Tags
        for tag in game.get('tags', []) or []:
            if not isinstance(tag, dict):
                continue
            local_tags_data.append({
                'game_id': game_id,
                'tag_id': tag.get('id', None),
                'tag_name': tag.get('name', ''),
                'tag_slug': tag.get('slug', ''),
                'tag_language': tag.get('language', '')
            })
        
        # 8. Ratings
        for rating in game.get('ratings', []) or []:
            if not isinstance(rating, dict):
                continue
            local_ratings_data.append({
                'game_id': game_id,
                'rating_id': rating.get('id', None),
                'rating_title': rating.get('title', ''),
                'rating_count': rating.get('count', 0),
                'rating_percent': rating.get('percent', 0)
            })
        
        # 9. Played status
        status = game.get('added_by_status', {}) or {}
        if not isinstance(status, dict):
            status = {}
        local_status_data.append({
            'game_id': game_id,
            'yet': status.get('yet', 0),
            'owned': status.get('owned', 0),
            'beaten': status.get('beaten', 0),
            'toplay': status.get('toplay', 0),
            'dropped': status.get('dropped', 0),
            'playing': status.get('playing', 0)
        })
        
        # 10. Metacritic by platform
        for platform_data in game.get('metacritic_platforms', []) or []:
            if not isinstance(platform_data, dict):
                continue
            platform = platform_data.get('platform', {}) or {}
            if not isinstance(platform, dict):
                platform = {}
            local_metacritic_platforms_data.append({
                'game_id': game_id,
                'platform_id': platform.get('platform', None),
                'platform_name': platform.get('name', ''),
                'platform_slug': platform.get('slug', ''),
                'metascore': platform_data.get('metascore', None),
                'metacritic_url': platform_data.get('url', '')
            })
            
        # 11. Derived metrics
        derived_metrics = calculate_derived_metrics(game, status)
        local_derived_metrics_data.append(derived_metrics)
        
        # Add local data to lock lists
        with data_lock:
            games_data.extend(local_games_data)
            platforms_data.extend(local_platforms_data)
            genres_data.extend(local_genres_data)
            stores_data.extend(local_stores_data)
            developers_data.extend(local_developers_data)
            publishers_data.extend(local_publishers_data)
            tags_data.extend(local_tags_data)
            ratings_data.extend(local_ratings_data)
            status_data.extend(local_status_data)
            metacritic_platforms_data.extend(local_metacritic_platforms_data)
            derived_metrics_data.extend(local_derived_metrics_data)
        
        return None
    
    except Exception as e:
        return f"Error processing {json_file}: {str(e)}"

## Create main function
1. Load the files
2. Create the paralel workers
3. Build dataframes
4. Save as CSVs

In [26]:
def main():
    start_time = time.time()
    print(f"Loading files...")
    
    # Get JSON list
    json_files = list(input_dir.glob('*.json'))
    total_files = len(json_files)
    
    if total_files == 0:
        print(f"There are no JSON files in input directory: {input_dir}")
        return
    
    print(f"Processing {total_files} files...")
    
    # Calculate number of workers
    max_workers = min(64, os.cpu_count() * 4 or 4)
    print(f"Using {max_workers} threads")
    
    # Process in paralel with progress bar
    errors = []
    with tqdm(total=total_files, desc="Processing files") as pbar:
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Map future files
            future_to_file = {executor.submit(process_json_file, file): file for file in json_files}
            
            # Get results while there are finishing
            for future in concurrent.futures.as_completed(future_to_file):
                file = future_to_file[future]
                try:
                    result = future.result()
                    if result:
                        errors.append(result)
                except Exception as exc:
                    errors.append(f"Unexpected error processing {file}: {exc}")
                
                # Update progress bar
                pbar.update(1)
    
    # Print errors
    if errors:
        print(f"\nFound {len(errors)} errors while processing:")
        for i, error in enumerate(errors[:10], 1):
            print(f"{i}. {error}")
        if len(errors) > 10:
            print(f"... and {len(errors) - 10} more errors.")
    
    # Verify we get some data
    if not games_data:
        print("Fatal error! No data found for games")
        return
    
    # Build DataFrames
    print("\nCreating DataFrames...")
    games_df = pd.DataFrame(games_data)
    platforms_df = pd.DataFrame(platforms_data)
    genres_df = pd.DataFrame(genres_data)
    stores_df = pd.DataFrame(stores_data)
    developers_df = pd.DataFrame(developers_data)
    publishers_df = pd.DataFrame(publishers_data)
    tags_df = pd.DataFrame(tags_data)
    ratings_df = pd.DataFrame(ratings_data)
    status_df = pd.DataFrame(status_data)
    metacritic_platforms_df = pd.DataFrame(metacritic_platforms_data)
    derived_metrics_df = pd.DataFrame(derived_metrics_data)
    
    # Save DataFrames as CSV
    print("Saving CSV files...")
    
    # CSV configuration
    csv_params = {
        'index': False,
        'quoting': csv.QUOTE_ALL,
        'escapechar': '\\',
        'doublequote': False,
        'encoding': 'utf-8',
        'lineterminator': '\n'
    }
    
    games_df.to_csv(output_dir / 'games.csv', **csv_params)
    platforms_df.to_csv(output_dir / 'game_platforms.csv', **csv_params)
    genres_df.to_csv(output_dir / 'game_genres.csv', **csv_params)
    stores_df.to_csv(output_dir / 'game_stores.csv', **csv_params)
    developers_df.to_csv(output_dir / 'game_developers.csv', **csv_params)
    publishers_df.to_csv(output_dir / 'game_publishers.csv', **csv_params)
    tags_df.to_csv(output_dir / 'game_tags.csv', **csv_params)
    ratings_df.to_csv(output_dir / 'game_ratings.csv', **csv_params)
    status_df.to_csv(output_dir / 'game_status.csv', **csv_params)
    metacritic_platforms_df.to_csv(output_dir / 'game_metacritic_platforms.csv', **csv_params)
    metacritic_platforms_df.to_csv(output_dir / 'game_metacritic_platforms.csv', **csv_params)
    derived_metrics_df.to_csv(output_dir / 'game_derived_metrics.csv', **csv_params)
    
    end_time = time.time()
    
    print("\nProcess finished")
    print(f"Execution time: {end_time - start_time:.2f} s")
    print(f"Processed files: {total_files - len(errors)}")
    print(f"CSV saved in: {output_dir}")
    
    # statistics
    print("\nStatistics:")
    print(f"Games: {len(games_df)}")
    print(f"Game-platform relations: {len(platforms_df)}")
    print(f"Game-genre relations: {len(genres_df)}")
    print(f"Game-store relations: {len(stores_df)}")
    print(f"Game-developer relations: {len(developers_df)}")
    print(f"Game-publisher relations: {len(publishers_df)}")
    print(f"Game-tag relations: {len(tags_df)}")
    print(f"Ratings: {len(ratings_df)}")
    print(f"Played status: {len(status_df)}")
    print(f"Reating by platform: {len(metacritic_platforms_df)}")
    print(f"Derived metrics: {len(derived_metrics_df)}")


In [27]:
main()

Loading files...
Processing 885166 files...
Using 64 threads


Processing files: 100%|██████████| 885166/885166 [06:01<00:00, 2451.21it/s]



Creating DataFrames...
Saving CSV files...

Process finished
Execution time: 390.63 s
Processed files: 885166
CSV saved in: ..\data\csv

Statistics:
Games: 885166
Game-platform relations: 1176534
Game-genre relations: 1043171
Game-store relations: 889724
Game-developer relations: 988390
Game-publisher relations: 234990
Game-tag relations: 4370881
Ratings: 119508
Played status: 885166
Reating by platform: 8118
Derived metrics: 885166
